## importing necessary libs

i'm using poetry for managing dependencies and my virtual env, so i'd recommend to look `pyproject.toml` to know what's been used here.

In [8]:
import findspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, Window, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt